In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
import random
from nltk.tokenize import RegexpTokenizer

In [2]:
# Load rating file
df_ratings = pd.read_csv('./ratings.dat', sep='::', engine='python', names=['user_id', 'item_id', 'rating', 'time_stamp'])
# Preprocess rating and gender
df_ratings['rating'] = df_ratings['rating'].map(lambda x: 0 if x < 4 else 1)

In [3]:
# Get records and sample slate for each user, slate length is set as 20
df_ratings = df_ratings.sort_values(by='time_stamp')
user_ids = list(set(df_ratings['user_id']))
user2records = df_ratings.groupby(['user_id'])
df_groups = []
for k, user_id in enumerate(user_ids):
    df_group = user2records.get_group(user_id)
    item_id_list = list(df_group['item_id'])
    rating_list = list(df_group['rating'])
    length = len(item_id_list)
    slate_id_c, slate_rating_c, slate_pos_c = [], [], []
    for i in range(length):
        low = max(i - 19, 0)
        high = min(i, length - 20)
        l = random.randint(low, high)
        r = l + 19
        slate_id_c.append(item_id_list[l: i] + item_id_list[i + 1: r + 1])
        slate_rating_c.append(rating_list[l: i] + item_id_list[i + 1: r + 1])
        slate_pos_c.append(i - l)
    df_group.insert(df_group.shape[1], 'slate_id', slate_id_c)
    df_group.insert(df_group.shape[1], 'slate_rating', slate_rating_c)
    df_group.insert(df_group.shape[1], 'slate_pos', slate_pos_c)
    df_groups.append(df_group)
res_df = pd.concat(df_groups)
with open('./movilens1M_slate_data.pkl', 'wb+') as f:
    pickle.dump(res_df, f)

In [4]:
res_df

,user_id,item_id,rating,time_stamp,slate_id,slate_rating,slate_pos
31,1,3186,1,978300019,"[1721, 1270, 1022, 2340, 1836, 3408, 1207, 280...","[1721, 1270, 1022, 2340, 1836, 3408, 1207, 280...",0
27,1,1721,1,978300055,"[3186, 1270, 1022, 2340, 1836, 3408, 1207, 280...","[1, 1270, 1022, 2340, 1836, 3408, 1207, 2804, ...",1
22,1,1270,1,978300055,"[3186, 1721, 1022, 2340, 1836, 3408, 1207, 280...","[1, 1, 1022, 2340, 1836, 3408, 1207, 2804, 260...",2
37,1,1022,1,978300055,"[1721, 1270, 2340, 1836, 3408, 1207, 2804, 260...","[1, 1, 2340, 1836, 3408, 1207, 2804, 260, 720,...",2
24,1,2340,0,978300103,"[1270, 1022, 1836, 3408, 1207, 2804, 260, 720,...","[1, 1, 1836, 3408, 1207, 2804, 260, 720, 1193,...",2
...,...,...,...,...,...,...,...
1000019,6040,2917,1,997454429,"[3168, 3182, 2575, 1212, 3362, 1258, 3421, 127...","[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...",19
999988,6040,1921,1,997454464,"[2575, 1212, 3362, 1258, 3421, 1270, 2068, 194...","[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",18
1000172,6040,1784,0,997454464,"[2575, 1212, 3362, 1258, 3421, 1270, 2068, 194...","[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",19
1000167,6040,161,0,997454486,"[3362, 1258, 3421, 1270, 2068, 1947, 1211, 257...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",18
